In [ ]:
import pandas as pd
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import MetaData, types
from sqlalchemy import create_engine
from sqlalchemy import Table, Column

In [ ]:
#Constantes
DB = "tp33"
HOST = "localhost"
PASS = "password"
USER = "postgres"


In [ ]:
#Creación de la Base de Datos

con = None
con = connect(user=USER,host=HOST,password=PASS)

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = con.cursor()
cur.execute('CREATE DATABASE '+ DB)
cur.close()
con.close()

In [ ]:
engine = create_engine('postgresql://'+USER+':'+PASS+'@'+HOST+':5432/'+DB)

metadata = MetaData()

customer = Table('alumno', metadata,
    Column('id', types.Integer, primary_key=True),         
    Column('nombre', types.String(length=50)),
    Column('apellido', types.String(length=50)),
    Column('email', types.String(length=50)),
    Column('sexo', types.String(length=50)),
    Column('direccion', types.String(length=50)),
    Column('pais', types.String(length=50)),
    Column('fecha_ingreso', types.Date), 
    Column('fecha_nacimiento', types.Date)
 )

metadata.create_all(engine)

In [ ]:
#Carga de la BD a traves de un .json
df_json = pd.read_json("dataTp3/dataJSON.json")
alumno = df_json.to_sql('alumno', engine, index=None,if_exists='append')


In [ ]:
#Lectura de datos
df_database = pd.read_sql_table('alumno',engine)
df_json = pd.read_json("dataTp3/dataJSON.json")
df_csv = pd.read_csv("dataTp3/dataCSV.csv")

CONSULTAS CON JOIN LEFT, OUTER, INNER
----------------------------------------

In [ ]:
#Seteo de indice para cada DF
df_database.set_index(['id'])
df_json.set_index(['id'])
df_csv.set_index(['id'])

#Slicing de las columnas de los DF
df_database = df_database[["nombre","apellido"]]
df_json = df_json[["email", "sexo", "direccion"]]
df_csv = df_csv[["pais", "fecha_ingreso", "fecha_nacimiento"]]

In [ ]:
resul_left = df_database.join(df_json)
resul_left = resul_left.join(df_csv)
resul_left

In [ ]:
resul_outer = df_database.join(df_json, how="outer")
resul_outer = resul_outer.join(df_csv, how="outer")
resul_outer

In [ ]:
resul_inner = df_database.head().join(df_json, how="inner")
resul_inner = resul_inner.join(df_csv,how="inner")
resul_inner

In [ ]:
resul_left.to_sql('join_left', engine)

MERGE
====



In [ ]:
resul = pd.merge(df_csv, df_json, how="inner", on=["pais","id"])
resul = pd.merge(resul, df_database, how="inner", on=["pais","id"])
resul.to_sql('merge', engine)

CONCAT
====





In [ ]:
result = pd.concat([df_json,df_csv,df_temp],keys=["json","csv","sql"])
result.ix["csv"]
result.ix["json"]
result.ix["sql"]
result.to_sql('concat', engine)

APPEND
=====

In [ ]:
result = df_json.append(df_csv, ignore_index=True)
result = result.append(df_database,ignore_index=True)
result.to_sql('append', engine)